# Skillbox — Мастер-класс: Разведочный анализ данных + Подготовка отчётов — Kiva.org
***

# Новый раздел

# 4. Домашнее задание
***

# Подготовка данных

In [2]:
import numpy as np
import pandas as pd
import math
import missingno as msno
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('Set2')

In [3]:
df_kiva_loans = pd.read_csv("/home/user/PycharmProjects/files/1-15-kiva/kiva_loans.csv")
df_mpi = pd.read_csv("/home/user/PycharmProjects/files/1-15-kiva/kiva_mpi_region_locations.csv")

In [4]:
countries = df_kiva_loans['country'].value_counts()[df_kiva_loans['country'].value_counts(normalize=True)> 0.005]
list_countries = list(countries.index)
df_mpi_grouped = df_mpi\
    .groupby(['ISO', 'country', 'world_region'])['MPI']\
    .mean()\
    .fillna(0)\
    .reset_index()
df_kiva_loans = df_kiva_loans.merge(df_mpi_grouped, how='left', on='country')
# regions = df_kiva_loans['world_region'].value_counts()
# regions_list = regions.index.to_list()
df_kiva_loans['borrower_genders'] = [elem if elem in ['female','male'] else 'group' for elem in df_kiva_loans['borrower_genders'] ]


In [5]:
df_gender_by_country = \
    df_kiva_loans[df_kiva_loans['country'].isin(countries.index.values)]\
    .groupby(['country', 'borrower_genders'])['borrower_genders']\
    .count()\
    .groupby(level=0).apply(lambda x: 100 * x / x.sum())\
    .unstack('borrower_genders')\
    .fillna(0)\

df_gender_by_country = df_gender_by_country[['male', 'group', 'female' ]]

В каких странах преобладают мужчины? Группа людей? (п. 2.2.2.)

In [8]:
print('Страны где преобладают мужчины:')
df_gender_by_country[(df_gender_by_country.male > df_gender_by_country.female) &
                     (df_gender_by_country.male > df_gender_by_country.group)
]\
    .sort_values(by='male', ascending=False)

Страны где преобладают мужчины:


borrower_genders,male,group,female
country,,,
Nigeria,87.134964,0.878058,11.986977
Palestine,64.601445,0.183666,35.214889
Uganda,48.701519,11.421776,39.876705
Mexico,34.767462,30.743773,34.488765


In [7]:
print('Страны где преобладают группы:')
df_gender_by_country[(df_gender_by_country.group> df_gender_by_country.female) &
                     (df_gender_by_country.group> df_gender_by_country.male)
]\
    .sort_values(by='group', ascending=False)


Страны где преобладают группы:


borrower_genders,male,group,female
country,,,
Mali,4.835066,91.836120,3.328815
Cambodia,5.368010,73.682398,20.949592
Guatemala,20.314637,62.640219,17.045144
Tanzania,10.346810,59.149262,30.503928
Ghana,16.095107,57.933242,25.971651
Paraguay,19.129631,55.053348,25.817021
Rwanda,22.865627,42.746845,34.387528


## 4.1. Самостоятельная работа
***

- А в каких странах преобладают мужчины? Группа людей? (п. 2.2.2.)
- Изучите несколько займов с очень большими суммами. Что можно сказать о назначении этих заявок? В каких странах они оформлены? Каков гендерный состав заявителей? (п. 2.4.2.)
- Попробуйте найти связь между странами (регионами), полом заёмщика и суммой заявки. Какие выводы можно сделать из этих данных? (п. 2.4.3.)

## 4.2. Домашняя работа
***

Расширьте наш отчёт исследованием следующих вопросов:

- Зависимость пола заёмщика от региона планеты, сектора экономики.
- Влияние такого атрибута займа, как **Количество кредиторов** (lender_count), на суммы и сроки займов.
- Влияние показателя **MPI** (многомерный индекс бедности) в разных странах на суммы займов и сроки погашения.
- Зависимости таких метрик, как «Сумма займа», «Срок займа», «Время финансирования заявки», «Ежемесячный платёж», в разрезе **макрорегионов**.